In [1]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yaelsuarez (warm-kanelbullar). Use `wandb login --relogin` to force relogin


True

In [2]:
from diametery.line_fit import LineFit
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [3]:
with wandb.init(project="diameterY-simple", job_type="test", mode="online") as run:
    run.config.n_measurements = 30
    run.config.step_size = 0.3
    #Download dataset 2D fibers
    dataset_artifact = run.use_artifact("simple-fibers:v0")
    dataset_dir = dataset_artifact.download("dataset_files")
    model = LineFit(run.config.n_measurements, run.config.step_size)
    rows = []
    pbar = tqdm(desc="images")
    for f in os.listdir(dataset_dir): 
        if f.startswith("test") and f.endswith(".npz"):
            file_path = os.path.join(dataset_dir, f)
            example = np.load(file_path)
            diameter_pred, mask_meas_lines = model.predict(example["x"])
            class_labels = ({0:'bg', 1:'measured_lines'})
            wandb_im = wandb.Image(example["x"], caption="masks_measured_lines", masks={
                            'measurements':{
                                'mask_data': mask_meas_lines,
                                'class_labels': class_labels  
                        }})
            rows.append(dict(
                measured_lines=wandb_im,
                d=example["d"],
                d_pred=diameter_pred,
                ))
            pbar.update()
    df = pd.DataFrame(rows)
    df["Error_abs"] =  abs((df["d_pred"] - df["d"])/ df["d"])
    mean_abs_error = df["Error_abs"].mean()
    artifact = wandb.Artifact("test_table", type="test-results")
    table = wandb.Table(dataframe=df)
    artifact.add(table, name="test-results")
    run.log_artifact(artifact)
    run.log(dict(mean_abs_error=mean_abs_error))
    

wandb: Downloading large artifact simple-fibers:v0, 802.80MB. 3842 files... Done. 0:0:0.5
images: 25it [00:01, 17.06it/s]/Users/carmenlopez/dev/diameterY/diametery/line_fit.py:124: RuntimeWarning: Mean of empty slice.
  calculated_diameter = np.array(diameters).mean()
/Users/carmenlopez/miniconda3/envs/diametery/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
images: 58it [00:03, 15.24it/s]/Users/carmenlopez/miniconda3/envs/diametery/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
images: 128it [00:07, 15.77it/s]

mean_abs_error,▁
mean_abs_error,0.01721


images: 128it [00:19, 15.77it/s]